# Installations 
install `diffusers` as well `scipy`, `ftfy` and `transformers`. `accelerate` to achieve much faster loading. The other packages are required to run the notebook.

In [1]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate
!pip3 install labml
!pip3 install labml-nn
!pip3 install pytorch-lightning
!pip install Pillow==9.0.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# clone the repository
!git clone https://github.com/kk-digital/kcg-ml-sd1p4.git
# move to the repo
%cd kcg-ml-sd1p4/

fatal: destination path 'kcg-ml-sd1p4' already exists and is not an empty directory.
/home/tuan/kcg/kcg-ml-sd1p4/notebooks/kcg-ml-sd1p4


# Text to Image

In [ ]:
# download model weights
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

--2023-05-20 23:17:14--  https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt
Resolving huggingface.co (huggingface.co)... 2600:9000:234a:3000:8:2a4d:8540:93a1, 2600:9000:234a:9600:8:2a4d:8540:93a1, 2600:9000:234a:3400:8:2a4d:8540:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:234a:3000:8:2a4d:8540:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4c/37/4c372b4ebb57bbd02e68413d4951aa326d4b3cfb6e62db989e529c6d4b26fb21/fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd-v1-4.ckpt%3B+filename%3D%22sd-v1-4.ckpt%22%3B&Expires=1684851419&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzRjLzM3LzRjMzcyYjRlYmI1N2JiZDAyZTY4NDEzZDQ5NTFhYTMyNmQ0YjNjZmI2ZTYyZGI5ODllNTI5YzZkNGIyNmZiMjEvZmU0ZWZmZjFlMTc0YzYyNzI1NmU0NGVjMjk5MWJhMjc5YjM4MTZlMzY0YjQ5ZjliZTJ

In [ ]:
"""
---
title: Generate images using stable diffusion with a prompt
summary: >
 Generate images using stable diffusion with a prompt
---

# Generate images using [stable diffusion](../index.html) with a prompt
"""

import os
from pathlib import Path

import torch

from labml import lab, monit
from stable_diffusion.latent_diffusion import LatentDiffusion
from stable_diffusion.sampler.ddim import DDIMSampler
from stable_diffusion.sampler.ddpm import DDPMSampler
from stable_diffusion.utils.model import load_model, save_images, set_seed


class Txt2Img:
    """
    ### Text to image class
    """
    model: LatentDiffusion

    def __init__(self, *,
                 checkpoint_path: Path,
                 sampler_name: str,
                 n_steps: int = 50,
                 ddim_eta: float = 0.0,
                 ):
        """
        :param checkpoint_path: is the path of the checkpoint
        :param sampler_name: is the name of the [sampler](../sampler/index.html)
        :param n_steps: is the number of sampling steps
        :param ddim_eta: is the [DDIM sampling](../sampler/ddim.html) $\eta$ constant
        """
        # Load [latent diffusion model](../latent_diffusion.html)
        self.model = load_model(checkpoint_path)
        # Get device
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        # Move the model to device
        self.model.to(self.device)

        # Initialize [sampler](../sampler/index.html)
        if sampler_name == 'ddim':
            self.sampler = DDIMSampler(self.model,
                                       n_steps=n_steps,
                                       ddim_eta=ddim_eta)
        elif sampler_name == 'ddpm':
            self.sampler = DDPMSampler(self.model)

    @torch.no_grad()
    def __call__(self, *,
                 dest_path: str,
                 batch_size: int = 3,
                 prompt: str,
                 h: int = 512, w: int = 512,
                 uncond_scale: float = 7.5,
                 ):
        """
        :param dest_path: is the path to store the generated images
        :param batch_size: is the number of images to generate in a batch
        :param prompt: is the prompt to generate images with
        :param h: is the height of the image
        :param w: is the width of the image
        :param uncond_scale: is the unconditional guidance scale $s$. This is used for
            $\epsilon_\theta(x_t, c) = s\epsilon_\text{cond}(x_t, c) + (s - 1)\epsilon_\text{cond}(x_t, c_u)$
        """
        # Number of channels in the image
        c = 4
        # Image to latent space resolution reduction
        f = 8

        # Make a batch of prompts
        prompts = batch_size * [prompt]

        # AMP auto casting
        with torch.cuda.amp.autocast():
            # In unconditional scaling is not $1$ get the embeddings for empty prompts (no conditioning).
            if uncond_scale != 1.0:
                un_cond = self.model.get_text_conditioning(batch_size * [""])
            else:
                un_cond = None
            # Get the prompt embeddings
            cond = self.model.get_text_conditioning(prompts)
            # [Sample in the latent space](../sampler/index.html).
            # `x` will be of shape `[batch_size, c, h / f, w / f]`
            x = self.sampler.sample(cond=cond,
                                    shape=[batch_size, c, h // f, w // f],
                                    uncond_scale=uncond_scale,
                                    uncond_cond=un_cond)
            # Decode the image from the [autoencoder](../model/autoencoder.html)
            images = self.model.autoencoder_decode(x)

        # Save images
        save_images(images, dest_path, 'txt_')

    # functions for pipeline
    @torch.no_grad()
    def generate_text_embeddings(self, prompt, batch_size=4, uncond_scale=7.5):
        """
        :param prompt: is the prompt to generate images with
        """
        # Make a batch of prompts
        prompts = batch_size * [prompt]

        # AMP auto casting
        with torch.cuda.amp.autocast():
            # In unconditional scaling is not $1$ get the embeddings for empty prompts (no conditioning).
            if uncond_scale != 1.0:
                un_cond = self.model.get_text_conditioning(batch_size * [""])
            else:
                un_cond = None
            # Get the prompt embeddings
            cond = self.model.get_text_conditioning(prompts)
        
        # return the embeddings
        return cond, un_cond
    
    @torch.no_grad()
    def generate_latent_space(self, cond, un_cond, batch_size=4, uncond_scale=7.5, h=512, w=512):
        """
        :param prompt: is the prompt to generate images with
        """
        # Number of channels in the image
        c = 4
        # Image to latent space resolution reduction
        f = 8

        # AMP auto casting
        with torch.cuda.amp.autocast():
            # [Sample in the latent space](../sampler/index.html).
            # `x` will be of shape `[batch_size, c, h / f, w / f]`
            x = self.sampler.sample(cond=cond,
                                    shape=[batch_size, c, h // f, w // f],
                                    uncond_scale=uncond_scale,
                                    uncond_cond=un_cond)
        
        # return the embeddings
        return x
    
    @torch.no_grad()
    def generate_image(self, x):
        """
        :param prompt: is the prompt to generate images with
        """
        # AMP auto casting
        with torch.cuda.amp.autocast():
            # Decode the image from the [autoencoder](../model/autoencoder.html)
            image = self.model.autoencoder_decode(x)
        
        # return the embeddings
        return image


In [ ]:
# create an instance of the class
sampler_name = 'ddim'
steps = 50
batch_size = 4
scale = 7.5
txt2img = Txt2Img(checkpoint_path='sd-v1-4.ckpt',
                      sampler_name=sampler_name,
                      n_steps=steps)

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

# Generate Text Embeddings

In [ ]:
# function to get the embeddings
def get_embeddings(prompt, batch_size=1, uncond_scale=7.5):
    # get the embeddings
    cond, un_cond = txt2img.generate_text_embeddings(prompt=prompt, batch_size=batch_size, uncond_scale=uncond_scale)
    # return the embeddings
    return cond, un_cond

In [ ]:
cond, un_cond = get_embeddings(prompt=prompt, batch_size=batch_size, uncond_scale=scale)

# Generate Latents

In [ ]:
# function to get the latent space
def get_latent_space(cond, un_cond, batch_size=4, uncond_scale=7.5, h=512, w=512):
    # get the embeddings
    x = txt2img.generate_latent_space(cond=cond, un_cond=un_cond, batch_size=batch_size, uncond_scale=uncond_scale, h=h, w=w)
    # return the embeddings
    return x

In [ ]:
x = get_latent_space(cond=cond, un_cond=un_cond, batch_size=batch_size, uncond_scale=scale, h=512, w=512)

# Decoding Latents

In [ ]:
# function to get the image
def get_image(x):
    image = txt2img.generate_image(x)
    return image

In [ ]:
image = get_image(x)

# Save the Image

In [ ]:
# function to save the image
def save_image(image, dest_path):
    # save the image
    save_images(image, dest_path, 'txt_')


In [ ]:
save_image(image, dest_path='./output_images')

In [ ]:
# show the first image
from PIL import Image
Image.open('./output_images/txt_00000.jpeg')